%%% Imports

In [8]:

from __future__ import annotations
from time import time
from QCRVPRDataset import QCRVPRDataset, QCRVPRSyncDataset
from VPRNetwork import VPRNetwork
from plotting import plot_confusion_matrix, plot_match_images, plot_gps
from constants import brisbane_event_traverses, qcr_traverses, brisbane_event_traverses_aliases
from utils import get_short_traverse_name

import os, sys
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib import animation
from scipy import signal
from datetime import datetime

import numpy as np

import lava.lib.dl.slayer as slayer
from lava.lib.dl.slayer.classifier import Rate

from constants import synced_times

In [9]:
# Training settings
from cgi import test


epochs = 50
batch_size = 10

# Network settings
input_size = 34
threshold = 1.0

# Data settings
train_traverse = qcr_traverses[0]
test_traverse = qcr_traverses[1]
train_name = get_short_traverse_name(train_traverse)
test_name = get_short_traverse_name(test_traverse)
num_places = 82
start_time = 0
place_gap = 2 #164/num_places # The streams run for approximately 164 seconds 
samples_per_sec = 1000
place_duration = 2
max_spikes = None

use_pre_synced_times = False
incorporate_speed = False

if use_pre_synced_times:
    num_places = len(synced_times[train_traverse])
    print(num_places)

# Sequencer settings
sequence_length = 3

# Plot settings
redo_plot = False
vmin = 0
vmax = 0.05

def transpose( matrix):
    if len(matrix) == 0:
        return []
    return [[matrix[i][j] for i in range(len(matrix))] for j in range(len(matrix[0]))]


# Make a folder for the trained network
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

# Use GPU
print(torch.cuda.is_available())
device = torch.device('cuda')

#---------------- Create the Network -----------------#

# Create the network
net = VPRNetwork(input_size, num_places, threshold=threshold).to(device)


True


In [10]:
# Load the data
if use_pre_synced_times:
    training_set = QCRVPRSyncDataset(train_traverse, train=True, place_duration = place_duration, samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size)
    if incorporate_speed:
        testing_set  = QCRVPRSyncDataset(test_traverse, train=False, place_duration = place_duration, samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size, training_duration=training_set.training_duration) # training_duration=training_set.training_duration
    else:
        testing_set  = QCRVPRSyncDataset(test_traverse, train=False, place_duration = place_duration, samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size)

else: 
    training_set = QCRVPRDataset(train_traverse, train=True, place_duration = place_duration, place_gap=place_gap, num_places=num_places, start_time=start_time,samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size)
    if incorporate_speed:
        testing_set  = QCRVPRDataset(test_traverse, train=False, place_duration = place_duration, place_gap=place_gap, num_places=num_places, start_time=start_time, samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size, relative_place_times=training_set.relative_place_times, training_duration=training_set.training_duration) # training_duration=training_set.training_duration
    else:
        testing_set  = QCRVPRDataset(test_traverse, train=False, place_duration = place_duration, place_gap=place_gap, num_places=num_places, start_time=start_time, samples_per_sec=samples_per_sec, max_spikes=max_spikes, subselect_num=input_size, relative_place_times=training_set.relative_place_times)

train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=True)

Loading training event streams ...


  0%|          | 0/1 [00:00<?, ?it/s]

Duration: 164.20s (which is 36141275 events)


/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['x'].loc[small_filt0x] = i
/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['y'].loc[small_filt0y] = i
/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162]
The number of training substreams is: 82


  0%|          | 0/1 [00:00<?, ?it/s]

Duration: 164.20s (which is 38023723 events)
[  0.           1.99999481   3.99998962   5.99998443   7.99997924
   9.99997406  11.99996887  13.99996368  15.99995849  17.9999533
  19.99994811  21.99994292  23.99993773  25.99993255  27.99992736
  29.99992217  31.99991698  33.99991179  35.9999066   37.99990141
  39.99989622  41.99989104  43.99988585  45.99988066  47.99987547
  49.99987028  51.99986509  53.9998599   55.99985471  57.99984952
  59.99984434  61.99983915  63.99983396  65.99982877  67.99982358
  69.99981839  71.9998132   73.99980801  75.99980283  77.99979764
  79.99979245  81.99978726  83.99978207  85.99977688  87.99977169
  89.9997665   91.99976132  93.99975613  95.99975094  97.99974575
  99.99974056 101.99973537 103.99973018 105.99972499 107.99971981
 109.99971462 111.99970943 113.99970424 115.99969905 117.99969386
 119.99968867 121.99968348 123.99967829 125.99967311 127.99966792
 129.99966273 131.99965754 133.99965235 135.99964716 137.99964197
 139.99963678 141.9996316  143.9

/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['x'].loc[small_filt0x] = i
/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_stream['y'].loc[small_filt0y] = i


The number of testing substreams is: 82


/home/n10223622/snn-event-vpr/src/QCRVPRDataset.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chopped_stream['t'] -= chop_start


In [11]:
# Define an optimiser 
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Training the network
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction='sum').to(device)

# Create a training assistant object
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)


if redo_plot:
    for i, (input, label) in enumerate(train_loader): # training loop
        output = assistant.train(input, label)

else: 
    for epoch in range(epochs):

        for i, (input, label) in enumerate(train_loader): # training loop
            output = assistant.train(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        for i, (input, label) in enumerate(test_loader): # training loop
            output = assistant.test(input, label)
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        if epoch%20 == 19: # cleanup display
            print('\r', ' '*len(f'\r[Epoch {epoch:2d}/{epochs}] {stats}'))
            stats_str = str(stats).replace("| ", "\n")
            print(f'[Epoch {epoch:2d}/{epochs}]\n{stats_str}')

        if stats.testing.best_accuracy:
            torch.save(net.state_dict(), trained_folder + '/network.pt')
        stats.update()
        stats.save(trained_folder + '/')
        net.grad_flow(trained_folder + '/')

[Epoch  0/50] Train loss =     1.03072 | Test  loss =     0.99810                        accuracy = 0.01220
Accuracy was none

Accuracy was none
[Epoch  1/50] Train loss =     0.79608 (min =     1.03072)    accuracy = 0.02439  | Test  loss =     1.22084 (min =     0.99810)    accuracy = 0.04878 (max = 0.01220)
Accuracy was none
[Epoch  2/50] Train loss =     0.79561 (min =     0.79608)    accuracy = 0.02439 (max = 0.02439) | Test  loss =     0.77570 (min =     0.99810)    accuracy = 0.08537 (max = 0.04878)
Accuracy was none
[Epoch  3/50] Train loss =     0.72567 (min =     0.79561)    accuracy = 0.13415 (max = 0.02439) | Test  loss =     0.77962 (min =     0.77570)    accuracy = 0.13415 (max = 0.08537)
Accuracy was none
[Epoch  4/50] Train loss =     0.71915 (min =     0.72567)    accuracy = 0.15854 (max = 0.13415) | Test  loss =     0.75741 (min =     0.77570)    accuracy = 0.12195 (max = 0.13415)
Accuracy was none
[Epoch  5/50] Train loss =     0.63892 (min =     0.71915)    accuracy

In [12]:
# import the best network during training 
net.load_state_dict(torch.load(trained_folder + '/network.pt'))
net.export_hdf5(trained_folder + '/network.net')

# Get the output for the input to each place
test_loader2  = DataLoader(dataset=testing_set , batch_size=batch_size, shuffle=False)
rate = []
labels = []
for i, (input, label) in enumerate(test_loader2):
    output = net(input.to(device)) # Get network output
    #guesses = assistant.classifier(output).cpu().data.numpy() # get the predictions 
    rate.extend(Rate.rate(output).cpu().data.numpy()) # Get the firing rates for each place 
    labels.extend(label.cpu().data.numpy()) # Get place labels


# # Get rates in percentages of total
# for i in range(num_places):
#     sum = np.sum(rate[i])
#     if sum != 0:
#         rate[i] = np.divide(rate[i],sum)

# Make confusion matrix annotations
accuracy = 0
matches = []
annotations = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(rate[qryIndex])
    matches.append(max_idx)
    annotations[max_idx][qryIndex] = 'x'
    if max_idx ==qryIndex:
        accuracy += 1


#--------------- Apply a sequencer ------------------#
I = np.identity(sequence_length)
conv = signal.convolve2d(rate, I, mode='same')
print(np.shape(conv))

# Make confusion matrix annotations
accuracy_s = 0
matches_with_seq = []
annotations_s = [['' for i in range(num_places)] for j in range(num_places)]
for qryIndex in range(num_places):
    max_idx = np.argmax(conv[qryIndex])
    matches_with_seq.append(max_idx)
    annotations_s[max_idx][qryIndex] = 'x'
    if max_idx == qryIndex:
        accuracy_s += 1



(82, 82)


In [13]:

#--------------- Save Results ------------------#

# Make new folder for results
from tracemalloc import start
from plotting import plot_gps


time_stamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
results_path = "./../results/" + time_stamp
os.mkdir(results_path)


# Save query and match images 
images_path = results_path + "/matched_images"
images_path_seq = results_path + "/matched_images_seq"
os.mkdir(images_path)
os.mkdir(images_path_seq)

print('help')
plot_match_images(images_path + "/", matches, training_set.place_images, testing_set.place_images)
plot_match_images(images_path_seq + "/", matches_with_seq, training_set.place_images, testing_set.place_images)

# Save the confusion matrices
confusion_path = results_path + "/confusion_matrices"
os.mkdir(confusion_path)
rate = transpose(rate)
conv = transpose(conv)
output_path = confusion_path + "/confusion_matrix" 
output_path_s = confusion_path + "/confusion_matrix_seq" 
plot_confusion_matrix(rate, labels, annotations, output_path, vmin, vmax)
plot_confusion_matrix(conv, labels, annotations_s, output_path_s, vmin, vmax)

# Save GPS map'
# gps_path = results_path + "/gps_locations"
# plot_gps(gps_path, training_set.place_locations, testing_set.place_locations)

# Save test settings and accuracy
accuracy = accuracy/num_places
accuracy_s = accuracy_s/num_places


help


<Figure size 1296x288 with 0 Axes>

<Figure size 1296x288 with 0 Axes>

In [14]:

log_path = results_path + "/log.txt"

if use_pre_synced_times:
    log_string  = """
    ---- DATA SETTINGS ---- 
    Training datasets = {}
    Testing datasets = {} 
    # Places = {}
    Using presynced times = True
    Place duration = {} [s]
    Samples per second = {} 
    Max spikes per sample = {}
    Incorporate speed = {}
    ---- NETWORK SETTINGS ---- 
    Input size = {}x{} 
    Threshold = {}
    ---- TRAINING SETTINGS ---- 
    Epochs = {}
    Batch size = {}
    ---- SEQUENCER SETTINGS ----
    Sequence lengtth = {} 
    ---- RESULTS ----
    Accuracy = {}
    Accuracy (sequencer) = {}
    """.format(train_name,test_name, num_places, place_duration,
                    samples_per_sec,max_spikes, incorporate_speed, input_size,input_size,threshold,epochs,
                    batch_size,sequence_length,accuracy,accuracy_s)

else: 
    log_string  = """
    ---- DATA SETTINGS ---- 
    Training datasets = {}
    Testing datasets = {} 
    # Places = {}
    Start time = {} [s]
    Place gap = {} [s]
    Place duration = {} [s]
    Samples per second = {} 
    Max spikes per sample = {}
    Incorporate speed = {}
    ---- NETWORK SETTINGS ---- 
    Input size = {}x{} 
    Threshold = {}
    ---- TRAINING SETTINGS ---- 
    Epochs = {}
    Batch size = {}
    ---- SEQUENCER SETTINGS ----
    Sequence lengtth = {} 
    ---- RESULTS ----
    Accuracy = {}
    Accuracy (sequencer) = {}
    """.format(train_name,test_name,num_places,start_time,place_gap,place_duration,
                    samples_per_sec,max_spikes, incorporate_speed, input_size,input_size,threshold,epochs,
                    batch_size,sequence_length,accuracy,accuracy_s)
f = open(log_path,'w')
f.write(log_string)
f.close()

print("The accuracy of the network is: " + str(accuracy))
print("The accuracy with a sequencer is: " + str(accuracy_s))

The accuracy of the network is: 0.7804878048780488
The accuracy with a sequencer is: 0.9512195121951219
